In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/Kimia'

/content/drive/MyDrive/Kimia


In [4]:
import os
import sys
import torch
import numpy as np
import importlib
from torchvision import transforms
from data_utils.ModelNetDataLoader import ModelNetDataLoader, load_data, PointSampler, Normalize, RandomNoise, RandomRotate, toTensor

In [5]:
if torch.cuda.is_available():
      device = torch.device('cuda:0')
      print('running on GPU')
else:
      device = torch.device('cpu')
      print('running on CPU')

running on GPU


# Dynamic Saliency Map 

**Please specify the path to the models based on your setup.**

For pointnet and pointnet2 run the following cells

In [ ]:
# BASE_DIR = os.path.dirname(os.path.abspath('/content/drive/MyDrive/Kimia/data.ipynb'))
# ROOT_DIR = BASE_DIR
# sys.path.append(os.path.join(ROOT_DIR, 'models'))

In [ ]:
# log_dir = 'pointnet2_checkpoint'
# experiment_dir  =  'log/classification/' + log_dir
# num_class = 40
# model_name = os.listdir(experiment_dir + '/logs')[0].split('.')[0]
# model = importlib.import_module(model_name)
# criterion = model.get_loss()
# classifier = model.get_model(num_class, normal_channel=False)
# classifier = classifier.cuda()
# criterion = criterion.cuda()
# checkpoint = torch.load(str(experiment_dir) + '/checkpoints/best_model.pth')
# classifier.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

For DGCNN run the following cell

In [6]:
# DGCNN
from DGCNN_cls import DGCNN, cal_loss
k = 20
emb_dims = 1024
dropout_p = 0.5
model = DGCNN(k, emb_dims, dropout_p, output_channels=40).to('cuda:0')
checkpoint = torch.load('log/DGCNN_checkpoint.pt')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

To load the data, run the following cell. Specify paths based on your setup.

In [7]:
main_path = '/content/drive/MyDrive/Kimia'
test_data_path = 'Data/test_data.npy'
test_label_path = 'Data/test_labels.npy'
test_X, test_y = load_data(main_path, test_data_path, test_label_path, mode='test')
test_X, test_y = test_X.astype(np.float32), test_y.astype(np.int64)
batch_size = test_X.shape[0]
default_transform = transforms.Compose([Normalize(), toTensor()])

test_dataset = ModelNetDataLoader(test_X, test_y, transform=default_transform)
testDataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=10)
for data in testDataLoader:
  break
points, labels = data
print(points.shape)
print(labels.shape)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([16, 1024, 3])
torch.Size([16])


In [13]:
def dynamic_saliency_map(model, points, labels, device, n=100, T=20, alpha=1):
    model.eval()
    saliency_points = np.zeros((points.shape[0], 3, n))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    points = points.transpose(1, 2).float()  # B * 3 * 1024
    a = int(n/T)
    for i in range(T):
        points = points.to(device)
        labels = labels.to(device)
        # computing the gradient of the loss with respect to the points
        points.requires_grad = True
        
        # pointnet, pointnet++
        # **************************
        # pred, trans_feat = classifier(points)
        # loss = criterion(pred, labels.long(), trans_feat)
        # **************************
        
        # dgcnn
        # **************************
        pred = model(points)
        loss = cal_loss(pred, labels.long())
        # ***************************
        
        loss.backward()
        grad = points.grad
        # saliency map calculations
        sphere_core = torch.median(points, axis=2, keepdims=True)  # B * 3 * 1

        sphere_r = torch.sqrt(torch.sum(torch.square(
            points - sphere_core.values), axis=1))  # B * 1024

        sphere_axis = points - sphere_core.values  # B * 3 * 1024

        before_map = torch.sum(torch.multiply(
            grad, sphere_axis), axis=1)  # B * 1024

        map = -torch.multiply(before_map,
                              torch.pow(sphere_r, alpha))  # B * 1024

        drop_indices = torch.topk(map, a).indices  # B * n/T

        tmp = torch.zeros((points.shape[0], 3, points.shape[2] - a))
        numpy_points = points.cpu().detach().numpy()
        numpy_indices = drop_indices.cpu().detach().numpy()
        for j in range(points.shape[0]):
            saliency_points[j][:, i *
                               a: (i + 1) * a] = numpy_points[j][:, numpy_indices[j]]
            tmp[j] = torch.from_numpy(
                np.delete(numpy_points[j], numpy_indices[j], axis=1))

        points = tmp.detach().clone()

    # returns the points after dropping the most important ones and the most important points
    return points, saliency_points

In [ ]:
i = False
j = 0
for data in testDataLoader:
    points, labels = data
    new_points, saliency_points = dynamic_saliency_map(model=model, points=points, labels=labels, device=device, n = 200, T = 40)
    if i is False:
        final_set = new_points.cpu().detach().numpy()
        i = True
    else:
        final_set = np.append(final_set, new_points, axis = 0)
    print(j)
    j+=1

In [15]:
to_save = final_set.transpose(0, 2, 1)
np.save('Data/drop200_dgcnn/test_data.npy', to_save)